In [99]:
import pandas as pd
import subprocess
import math
import numpy as np
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools


In [213]:
translate_table={1:"act",2:"mul",3:"add", 4:"element-sum", 5:"sqrt", 6:"vec"}
GELU= [3,2,2,3,2,1,3,2,2]
Layer_norm= [3,3,4,2,3,3,2,4,2,5,6,2,3]
Score_cal=[3,2,2,1,4,6,2]

In [214]:
def MFU_chain(num_MFU=2,MFU_order=[1,2,3,4,5,6],vector_op_dict={}):
    #print(vector_op_dict)
    result=dict()
    total_chain=0
    for key, value in vector_op_dict.items():
        vector_op=value
        MFU_unit_idx=0
        MFU_idx=0
        chain_idx=1
        #print(vector_op)
        for i in range(len(vector_op)):
            next_op=vector_op[i]
            if(MFU_unit_idx<MFU_order[next_op-1]):
                MFU_unit_idx=MFU_order[next_op-1]
                #print("chain idx:",chain_idx," MFU_idx",MFU_idx, "-",MFU_unit_idx,":",translate_table[next_op])
            else:
                MFU_idx=MFU_idx+1
                MFU_unit_idx=next_op
                if(MFU_idx==num_MFU):
                    chain_idx=chain_idx+1
                    MFU_idx=0
                #print("chain idx:",chain_idx," MFU_idx",MFU_idx, "-",MFU_unit_idx,":",translate_table[next_op])
        result[key]=chain_idx
        total_chain=chain_idx+total_chain
        #print(key,":",chain_idx)
        
    return result,total_chain

In [215]:
def print_MFU_chain(num_MFU=2,MFU_order=[1,2,3,4,5,6],vector_op_dict={}):
    print("MFU unit order")
    for i in range(len(MFU_order)):
        for j in range(len(MFU_order)):
            if(i==MFU_order[j]):
                print(translate_table[MFU_order[j]])
    result=dict()
    total_chain=0
    for key, value in vector_op_dict.items():
        vector_op=value
        MFU_unit_idx=0
        MFU_idx=0
        chain_idx=1
        print(key, "start")
        for i in range(len(vector_op)):
            next_op=vector_op[i]
            if(MFU_unit_idx<MFU_order[next_op-1]):
                MFU_unit_idx=MFU_order[next_op-1]
                print("chain idx:",chain_idx," MFU_idx:",MFU_idx, "-",translate_table[next_op])
            else:
                MFU_idx=MFU_idx+1
                MFU_unit_idx=next_op
                if(MFU_idx==num_MFU):
                    chain_idx=chain_idx+1
                    MFU_idx=0
                print("chain idx:",chain_idx," MFU_idx",MFU_idx, "-",translate_table[next_op])
        result[key]=chain_idx
        total_chain=chain_idx+total_chain
        print(key," end")
        #print(key,":",chain_idx)
        
    return result,total_chain

In [216]:
def find_optimal_order(num_MFU=2):
    min_chain=1000000
    min_chain_list=[]
    permutation_list=list(itertools.permutations([1, 2, 3,4]))
    for permutation in permutation_list:
        MFU_order_table=np.array(permutation+(5,6))
        result,total_chain=MFU_chain(num_MFU,MFU_order_table,vector_op_dict)
        if(total_chain==min_chain):
            print(MFU_order_table)
            print (result)
        elif(total_chain<min_chain):
            print(MFU_order_table)
            print (result)
            min_chain=total_chain
            #calculate total MFU chaint

In [217]:
vector_op_dict={"GELU":[3,2,2,3,2,1,3,2,2],"Layer_norm":[3,3,4,2,3,3,2,4,2,5,6,2,3],"Score_cal":[3,2,2,1,4,6,2]}
find_optimal_order(num_MFU=3)

[1 2 3 4 5 6]
{'Layer_norm': 3, 'GELU': 3, 'Score_cal': 2}
[1 2 4 3 5 6]
{'Layer_norm': 3, 'GELU': 3, 'Score_cal': 2}
[1 3 2 4 5 6]
{'Layer_norm': 3, 'GELU': 2, 'Score_cal': 2}
[1 3 4 2 5 6]
{'Layer_norm': 3, 'GELU': 2, 'Score_cal': 2}
[1 4 3 2 5 6]
{'Layer_norm': 3, 'GELU': 2, 'Score_cal': 2}
[2 3 4 1 5 6]
{'Layer_norm': 3, 'GELU': 2, 'Score_cal': 2}
[2 4 3 1 5 6]
{'Layer_norm': 3, 'GELU': 2, 'Score_cal': 2}
[3 1 4 2 5 6]
{'Layer_norm': 3, 'GELU': 2, 'Score_cal': 2}
[3 2 1 4 5 6]
{'Layer_norm': 3, 'GELU': 3, 'Score_cal': 1}
[3 2 4 1 5 6]
{'Layer_norm': 3, 'GELU': 2, 'Score_cal': 2}


In [218]:
print_MFU_chain(3,[1,2,3,4,5,6],vector_op_dict)

MFU unit order
act
mul
add
element-sum
sqrt
Layer_norm start
chain idx: 1  MFU_idx: 0 - add
chain idx: 1  MFU_idx 1 - add
chain idx: 1  MFU_idx: 1 - element-sum
chain idx: 1  MFU_idx 2 - mul
chain idx: 1  MFU_idx: 2 - add
chain idx: 2  MFU_idx 0 - add
chain idx: 2  MFU_idx 1 - mul
chain idx: 2  MFU_idx: 1 - element-sum
chain idx: 2  MFU_idx 2 - mul
chain idx: 2  MFU_idx: 2 - sqrt
chain idx: 2  MFU_idx: 2 - vec
chain idx: 3  MFU_idx 0 - mul
chain idx: 3  MFU_idx: 0 - add
Layer_norm  end
GELU start
chain idx: 1  MFU_idx: 0 - add
chain idx: 1  MFU_idx 1 - mul
chain idx: 1  MFU_idx 2 - mul
chain idx: 1  MFU_idx: 2 - add
chain idx: 2  MFU_idx 0 - mul
chain idx: 2  MFU_idx 1 - act
chain idx: 2  MFU_idx: 1 - add
chain idx: 2  MFU_idx 2 - mul
chain idx: 3  MFU_idx 0 - mul
GELU  end
Score_cal start
chain idx: 1  MFU_idx: 0 - add
chain idx: 1  MFU_idx 1 - mul
chain idx: 1  MFU_idx 2 - mul
chain idx: 2  MFU_idx 0 - act
chain idx: 2  MFU_idx: 0 - element-sum
chain idx: 2  MFU_idx: 0 - vec
chain id

({'GELU': 3, 'Layer_norm': 3, 'Score_cal': 2}, 8)